<a href="https://colab.research.google.com/github/pavi-ninjaac/Specrom_Analysis/blob/main/Internship_works/week2/test_textclass_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
#import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#sklearn 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn. preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score

In [75]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/Text_classification_news_article/data_news_cleaned.csv')
print("the shape of the dataset----->",data.shape)

the shape of the dataset-----> (5722, 16)


In [72]:
#keep only needed columns
column_keep = ['author','description','full_text','title','topic']
data = data[column_keep]
data['topic'].value_counts()

business         1387
tech             1331
entertainment    1182
sport            1118
politics          704
Name: topic, dtype: int64

In [76]:
data['text'] = data['description'] + '-' + data['full_text']
data = data[['text','topic']]
data.dropna(axis=0,how='any',inplace=True)
data.head(4)

,text,topic
0,The Food and Drug Administration has offered a...,politics
1,The billâs gifts to the wealthy underscore t...,politics
2,It is naive to think that either party is free...,tech
3,Much of the loudest opposition to the $600 che...,business


# Vectorization 

In [30]:
X_test , y_test = data['text'] , data['topic']
# label ecode the terget value
label = LabelEncoder()
y_test = label.fit_transform(y_test)

In [31]:
#ifidf vectorizer

vecto =  TfidfVectorizer(stop_words='english',max_df = 0.99,min_df=10,
                                   ngram_range=(1, 2),lowercase=True, max_features=5000)
vecto = vecto.fit(X_test)

X_test = vecto.transform(X_test).toarray()
X_test.shape

(5707, 5000)

#predict the labels using the pretrained logistric Regression model

In [34]:
def classifier_scores(X,y, y_pred):
    
    print()
       
    print("Test data accuracy score: ", accuracy_score(y,y_pred))
    print()
    print("Recall score on test data: ",recall_score( y,y_pred, average='macro'))
    print()
    print("Precision score on test data: ",precision_score(y,y_pred, average='macro'))
    print()
    print("F1 score on test data: ",f1_score(y,y_pred, average='macro'))

In [53]:
#load the model 
file_path = '/content/drive/MyDrive/Colab Notebooks/dataFiles/Text_classification_news_article/logistricRegression_text_classi.sav'
log_reg = pickle.load(open(file_path , 'rb'))
y_pred = log_reg.predict(X_test)
classifier_scores(X_test , y_test , y_pred)


Test data accuracy score:  0.08077799193972315

Recall score on test data:  0.026904737605972815

Precision score on test data:  0.0628267853311363

F1 score on test data:  0.03626773332591833


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
#label the dataset 
classes = ['POLITICS',
 'WELLNESS',
 'ENTERTAINMENT',
 'TRAVEL',
 'STYLE & BEAUTY',
 'PARENTING',
 'HEALTHY LIVING',
 'QUEER VOICES',
 'FOOD & DRINK',
 'BUSINESS',
 'COMEDY',
 'PARENTS',
 'SPORTS',
 'HOME & LIVING',
 'BLACK VOICES',]

def num_label(row):
  
  return classes[row[0]]
y_pred = pd.DataFrame(y_pred)
y_pred['label'] = y_pred.apply(lambda row: num_label(row) , axis=1)

#data_labeled = pd.concat(data['text'],[])

In [80]:
data['predicted_label'] = y_pred['label']
data.head(10)

,text,topic,predicted_label
0,The Food and Drug Administration has offered a...,politics,ENTERTAINMENT
1,The billâs gifts to the wealthy underscore t...,politics,PARENTS
2,It is naive to think that either party is free...,tech,HOME & LIVING
3,Much of the loudest opposition to the $600 che...,business,BLACK VOICES
4,"Reprising its 2016 and 2018 prison strikes, Fr...",tech,WELLNESS
5,The county of Los Angeles is perilously close ...,business,WELLNESS
6,What Trump may do in his waning days is only u...,politics,COMEDY
7,Critics say the $900 billion relief package do...,business,COMEDY
8,"All the fashion, design, and art books that tr...",entertainment,SPORTS
9,Modeling is a physical business. So what happe...,entertainment,POLITICS
